In [ ]:
from heatmap_model.interaction_model import UQnet,TrajModel
import numpy as np
import sys
import matplotlib.pyplot as plt
%matplotlib inline 

import torch
from torch import nn, Tensor
import torchvision.datasets as dataset
from torch.optim.lr_scheduler import StepLR
import datetime

from scipy.special import expit

from heatmap_model.utils import *
from heatmap_model.train import *
from heatmap_model.interaction_dataset import *
from heatmap_model.losses import *
from config import *

from absl import logging
logging._warn_preinit_stderr = 0
logging.warning('Worrying Stuff')

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

In [ ]:
# change hyper-parameters here
paralist['resolution'] = 1. # change to any value during test
paralist['encoder_attention_size'] = 128
paralist['use_sem'] = False
paralist['epoches'] = 8
paralist['mode'] = 'lanescore'
paralist['prob_mode'] = 'ce'
paralist['inference'] = False # change to True during test
paralist['batch_size'] = 16

In [ ]:
# set test=True during test, drivale is optional
model = UQnet(paralist, test=False, drivable=False).to(device)
#testset = InteractionDataset(['val'], 'val', paralist, mode=paralist['mode'], filters=False)
#testset = InteractionDataset(['test'], 'test', paralist, mode=paralist['mode'], filters=False)

In [ ]:
# validation set
validationset = InteractionDataset(['val'], 'val', paralist, paralist['mode'], filters=False)
validation_loader = DataLoader(validationset, batch_size=paralist['batch_size'], shuffle=False)

In [ ]:
BATCH_SIZE = paralist['batch_size']
EPOCH_NUMBER = paralist['epochs']
loss = OverAllLoss(paralist).to(device)

In [ ]:
# train set
trainset = InteractionDataset(['train1', 'train2','train3','train4'], 'train', paralist, paralist['mode'], filters=False)

In [ ]:
# train 7 models to compose an ensemble
for order in range(7):
    model = UQnet(paralist).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler_heatmap = StepLR(optimizer, step_size=1, gamma=0.975)
    train_model(EPOCH_NUMBER, BATCH_SIZE, trainset, model, optimizer, validation_loader, loss,
                  scheduler_heatmap, paralist, mode=paralist['mode'])
    torch.save(model.encoder.state_dict(), './interactionDE/encoder'+str(order)+'.pt')
    torch.save(model.decoder.state_dict(), './interactionDE/decoder'+str(order)+'.pt')

In [ ]:
M, T, Nm, Na, Yp, Ua, Um, H, Y = inference_model(model, './interactionDE/', 'val', 1, testset, paralist, 
                                              k=6, test=False, return_heatmap=True, mode=paralist['mode'], batch=4)

In [ ]:
FDE = ComputeError(Yp,Y, r=1, sh=6)